<a href="https://colab.research.google.com/github/dasnikita/Data-Science-work/blob/main/Handwritten_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets download dhruvildave/english-handwritten-characters-dataset

!unzip "/content/english-handwritten-characters-dataset.zip" -d "/content/data"

In [2]:
!nvidia-smi

Mon Jan 10 08:52:54 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8    27W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

**TechVidvan**

In [1]:
#importing libraries
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Flatten,Conv2D, MaxPooling2D, MaxPool2D, Dropout
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder 
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator


%matplotlib inline

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip "/content/drive/MyDrive/Handwritten/Img.zip" -d "/content/Images"

In [6]:
#setting path for home directory
home_dir = r"/content/drive/MyDrive/Handwritten"

In [7]:
df = pd.read_csv(home_dir + r'/english.csv')
df.head()

,image,label
0,Img/img001-001.png,0
1,Img/img001-002.png,0
2,Img/img001-003.png,0
3,Img/img001-004.png,0
4,Img/img001-005.png,0


In [8]:
temp_path = ['/content/Images/'+x for x in df['image'].values]
df['image'] = temp_path

In [10]:
#acessing meta data provided in the .csv file
from sklearn.preprocessing import MinMaxScaler
x = df['image']
y = df['label']
enc = LabelEncoder()  # encoding classes
enc.fit(y)
y_encoded = enc.transform(y)

In [11]:
X_train, X_test, Y_train, Y_test = train_test_split(x, y_encoded, test_size = 0.2, random_state = 1)

In [12]:
traindf = pd.DataFrame(data=list(zip(X_train,Y_train)),columns=['path','label'])
testdf = pd.DataFrame(data=list(zip(X_test,Y_test)),columns=['path','label'])

In [13]:
traindf['label'] = traindf['label'].astype(str)
testdf['label'] = testdf['label'].astype(str)

In [15]:
# using VGG16 model in tensorflow keras
tf.keras.backend.clear_session()
IMAGE_SIZE = [224,224]
input_layer = tf.keras.layers.Input(shape=(224,224,3))
vgg = tf.keras.applications.vgg16.VGG16(input_tensor=input_layer, weights='imagenet', include_top=False)
for layers in vgg.layers:
  layers.trainable = False


58900480/58889256 [==============================] - 0s 0us/step


In [16]:
#using ImageDataGenerator and flow_from_dataframe for generating image data
datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1/255)
train = datagen.flow_from_dataframe(dataframe=traindf,x_col = 'path',y_col='label',color_mode='rgb',target_size=IMAGE_SIZE,class_mode="categorical")
test = datagen.flow_from_dataframe(dataframe=testdf,x_col = 'path',y_col='label',color_mode='rgb',target_size=IMAGE_SIZE, class_mode="categorical")

Found 2728 validated image filenames belonging to 62 classes.
Found 682 validated image filenames belonging to 62 classes.


In [17]:
# defining our model
conv_layer = keras.layers.Conv2D(filters=64, kernel_size=(3,3))(vgg.output)
maxpool_layer = keras.layers.MaxPool2D(pool_size=(2, 2), strides=None, padding="valid")(conv_layer)
flatten = keras.layers.Flatten()(maxpool_layer)
fc1 = keras.layers.Dense(1024,activation='relu')(flatten)
fc2 = keras.layers.Dense(768,activation='relu')(fc1)
fc3 = keras.layers.Dense(512,activation='relu')(fc1)
fc4 = keras.layers.Dense(256,activation='relu')(fc3)
fc5 = keras.layers.Dense(128,activation='relu')(fc4)
fc6 = keras.layers.Dense(112,activation='relu')(fc5)
output = keras.layers.Dense(62,activation='softmax')(fc6)

In [18]:
model1 = keras.models.Model(vgg.input,output)

model1.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)
model1.fit(train, validation_data=test,epochs = 10,batch_size = 128)

Epoch 1/10
86/86 [==============================] - 87s 742ms/step - loss: 3.5406 - accuracy: 0.1210 - val_loss: 2.3188 - val_accuracy: 0.3328
Epoch 2/10
86/86 [==============================] - 55s 633ms/step - loss: 1.5996 - accuracy: 0.5139 - val_loss: 1.1193 - val_accuracy: 0.6349
Epoch 3/10
86/86 [==============================] - 54s 632ms/step - loss: 0.8725 - accuracy: 0.7199 - val_loss: 0.9052 - val_accuracy: 0.7155
Epoch 4/10
86/86 [==============================] - 54s 631ms/step - loss: 0.5607 - accuracy: 0.8284 - val_loss: 0.7202 - val_accuracy: 0.7727
Epoch 5/10
86/86 [==============================] - 55s 633ms/step - loss: 0.4457 - accuracy: 0.8508 - val_loss: 0.6737 - val_accuracy: 0.7903
Epoch 6/10
86/86 [==============================] - 54s 631ms/step - loss: 0.3752 - accuracy: 0.8779 - val_loss: 0.7742 - val_accuracy: 0.7977
Epoch 7/10
86/86 [==============================] - 54s 630ms/step - loss: 0.3909 - accuracy: 0.8717 - val_loss: 0.7883 - val_accuracy: 0.7845

In [19]:
model1.evaluate(test,batch_size=128)

22/22 [==============================] - 11s 487ms/step - loss: 0.7355 - accuracy: 0.8079


[0.7354844808578491, 0.8079178929328918]